# HOG 전처리 기법 추가 


In [1]:
import pandas as pd
import numpy as np

import sys
# sys.path.append("C:/Users/hk-Lee/AppData/Local/Programs/Python/Python310/Lib/site-packages")
# sys.path
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.decomposition import PCA
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

import os
from tqdm import tqdm
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



In [2]:
training_data = pd.read_csv('C:/Users/유재림/Desktop/ML23_2-main/archive/fashion-mnist_train.csv')

train_y = training_data['label']

train_X = training_data.drop('label',axis=1)


In [3]:
datagen = ImageDataGenerator(
    width_shift_range=0.10,
    height_shift_range=0.10
)

np.random.seed(42)

# Data augmentation and adding to the training set loop
aug_train_X = []
aug_train_y = []

for index, row in tqdm(train_X.iterrows(), total=len(train_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 1))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    aug_train_X.append(img_array[0])
    aug_train_y.append(train_y[index])

    # Augmented data with 2/3 probability
    
    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])

print(len(aug_train_X))
print(len(aug_train_y))


100%|██████████| 60000/60000 [00:19<00:00, 3019.33it/s]

120000
120000


In [4]:
for index, row in tqdm(train_X.iterrows(), total=len(train_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 1))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    # aug_train_X.append(img_array[0])
    # aug_train_y.append(train_y[index])

    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])
    
# Convert the lists to numpy arrays
aug_train_X = np.array(aug_train_X)
aug_train_y = np.array(aug_train_y)

# Check the shape of the new arrays
print("Shape of aug_train_X:", aug_train_X.shape)
print("Shape of aug_train_y:", aug_train_y.shape)


100%|██████████| 60000/60000 [00:17<00:00, 3367.05it/s]


Shape of aug_train_X: (180000, 28, 28, 1)
Shape of aug_train_y: (180000,)


# 호그 특징 처리 


In [5]:
from skimage.feature import hog
from skimage import exposure

# Function to apply HOG transformation and combine with original features
def apply_combined_feature_extraction(images, pca_components=256):
    combined_features = []
    for img in images:
        # Reshape the image to its original shape after PCA
        img_reshaped = img.reshape((int(np.sqrt(pca_components)), int(np.sqrt(pca_components))))

        # Apply HOG transformation
        _, hog_feature = hog(img_reshaped, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

        # 호그 특징 펼친거 
        hog_feature_flattened = hog_feature.flatten()

        # 오리지널 특징 
        original_feature_flattened = img.flatten()

        # 원래 특징 + 호그 특징 
        combined_feature = np.concatenate((original_feature_flattened, hog_feature_flattened))

        combined_features.append(combined_feature) # 혼합 특징 

    return np.array(combined_features)

# Assuming aug_train_X is your original dataset
aug_train_X = aug_train_X.reshape((aug_train_X.shape[0], -1))
aug_train_X /= 255.0

# Apply combined feature extraction
aug_train_X_combined = apply_combined_feature_extraction(aug_train_X)

# Now, aug_train_X_combined contains both original and HOG features


c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
print(aug_train_X.shape)

(180000, 784)


In [7]:
pca = PCA(n_components=256)
aug_train_X_pca = pca.fit_transform(aug_train_X)

# Apply HOG transformation to PCA features
aug_train_X_hog_pca = apply_hog_transform(aug_train_X_pca)

# Combine PCA and HOG features
aug_train_X_combined = np.hstack((aug_train_X_pca, aug_train_X_hog_pca))

# Normalize the combined features
scaler = StandardScaler()
aug_train_X_combined = scaler.fit_transform(aug_train_X_combined)

In [8]:
svc = SVC(gamma='scale', kernel='rbf', C=13)
svc.fit(aug_train_X_combined, aug_train_y)

In [ ]:
test_data_folder = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/data"

# 테스트 데이터 로드 및 전처리
test_X = []
file_names = []

print(len(os.listdir(test_data_folder)))
# tqdm으로 래핑
for file_name in tqdm(os.listdir(test_data_folder), desc="Loading and preprocessing"):
    if file_name.endswith(".png"):
    
        file_path = str(test_data_folder) + "/" + str(file_name)
        try:

            img_array = np.fromfile(file_path, np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
            image_array = img / 255.0  # 0부터 1사이의 값
            test_X.append(image_array.flatten())  # 2D 배열을 1D로 펼침
            
            file_names.append(file_name)
        except Exception as e:
            print(f"Error processing file {file_name}: {str(e)}")


# NumPy 배열로 변환
test_X = np.array(test_X)
print(test_X.shape)

if len(test_X) == 0:
    print("No valid test images found.")
else:
    print("pca 적용")
    test_X_pca = pca.transform(test_X)
    print(test_X_pca.shape)
    predictions = svc.predict(test_X_pca)

with open('testResult3.txt', 'w') as file:
    for i, pred in enumerate(predictions):
        file.write(f"{i:05d} {pred}\n")


In [ ]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

testResult_path = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/testResult3.txt"
label_path = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/label.txt"

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    AP.append(auc(temp_recall, temp_precision))
    
mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.4f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.4f} |\n".format("mAP", mAP)

print(table)